In [5]:
#2번
# 비 유무 예측 주제로 R을 이용한 로지스틱 회귀분석을 실시하고, 인공신경망을 
# 이용한 로지스틱 회귀분석을 python으로 실행하여 결과를 비교한다.
library(car)
library(caret)
library(ModelMetrics)
library(corrplot)
df<-read.csv("E:/GoogleDrive/절대삭제노노/DATASET/dataset4/weather.csv")

#EDA
head(df)
str(df)
summary(df)

df<-na.omit(df) #결측치 제거
cordf<-df[sapply(df,is.numeric)]  #numeric컬럼만 cordf에 넣기
M <- cor(cordf)
corrplot(M, method="circle")
#Temp와minTemp, maxTemp 상관관계가 매우 크다. 다중공선성 가능성 큼

#결측치 제거, 전처리
df<-df[,-1] #날짜 제거
df<-na.omit(df) #결측치 제거
p<-preProcess(df,"range") #모든 숫자형변수 범위0~1 최소-최대 정규화
df<-predict(p,df)

#모델 생성후 다중 공선성 문제로 다시 전처리 부분
df<-df[,-c(1,2,5,7)]
##전처리 후 다중공선성 
# Rainfall      Sunshine WindGustSpeed     WindSpeed      Humidity      Pressure         Cloud 
# FALSE         FALSE         FALSE         FALSE         FALSE         FALSE         FALSE 
# Temp     RainToday 
# FALSE         FALSE 


#모델생성
v <- 1:nrow(df)
s <- SyncRNG(seed=42)
idx <- s$shuffle(v)[1:round(nrow(df)*0.7)]

train<-df[idx,]
test<-df[-idx,]
m_glm<-glm(RainTomorrow~.,train,family = 'binomial')
# Warning message:
# glm.fit: 적합된 확률값들이 0 또는 1 입니다 
# 원인 다중공선성
# 다중공선성 변수 제거해야함
vif(m_glm)
summary(m_glm)


##유의한 변수 선택
#후진제거법으로 로지스틱회귀모형을 새롭게 정의
m_glm=step(m_glm, direction = "backward")
summary(m_glm)

p_glm<-predict(m_glm,test,type='response')
p.glm<-ifelse(p_glm>=0.5,'Yes','No')

#모델평가
table(test$RainTomorrow)
table(p.glm)

caret::confusionMatrix(as.factor(p.glm),test$RainTomorrow)
caret::confusionMatrix(as.factor(p.glm),test$RainTomorrow)$byClass
caret::confusionMatrix(as.factor(p.glm),test$RainTomorrow)$overall
auc(as.factor(p.glm),test$RainTomorrow)



 No Yes 
 89  19 

ERROR: Error in table(p.glm): 객체 'p.glm'를 찾을 수 없습니다
